<a href="https://colab.research.google.com/github/1SenaMetth/Processamento-de-Imagens_E02_Grupo2/blob/main/Grupo_2_Convers%C3%A3o_de_imagens_para_escala_de_cinza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importação das bibliotecas necessárias

In [ ]:
import cv2 as cv
import numpy as np
import time
import matplotlib.pyplot as plt
import ipywidgets as widgets
from google.colab import files
from IPython.display import display, clear_output
import io
import zipfile
import os

# Célula 1: Funções de Conversão e Transformação

In [ ]:
def converter_para_cinza(img_bgr):
    """Converte uma imagem BGR para escala de cinza usando o método de luminosidade."""
    return cv.cvtColor(img_bgr, cv.COLOR_BGR2GRAY)

def aplicar_transformacao(img_cinza, metodo):
    """Aplica transformações de contraste na imagem em escala de cinza."""
    img_float = img_cinza.astype(np.float64)

    if metodo == 'Logaritmo':
        denominador = np.log(1 + np.max(img_float))
        if denominador == 0:
            return img_cinza
        c = 255 / denominador
        img_transformada = c * np.log(1 + img_float)
    elif metodo == 'Quadrado':
        img_norm = img_float / 255.0
        img_transformada = np.power(img_norm, 2.0) * 255
    elif metodo == 'Exponencial':
        img_norm = img_float / 255.0
        img_transformada = np.power(img_norm, 0.5) * 255
    else:
        return img_cinza
    return np.uint8(img_transformada)

def inverter_cores(img_cinza):
    """Inverte as cores da imagem (efeito negativo)."""
    return cv.bitwise_not(img_cinza)


def exibir_canal_cor(img_bgr, canal):
    """Exibe um canal de cor específico (R, G, B) da forma mais simples."""
    azul, verde, vermelho = cv.split(img_bgr)
    if canal == 'Vermelho':
        return vermelho
    elif canal == 'Verde':
        return verde
    elif canal == 'Azul':
        return azul
    else:
        return cv.cvtColor(img_bgr, cv.COLOR_BGR2GRAY)

def plotar_histograma(img_cinza):
    """Calcula e plota o histograma de uma imagem em escala de cinza."""
    plt.figure(figsize=(6, 4))
    plt.title("Histograma de Tons de Cinza")
    plt.xlabel("Intensidade")
    plt.ylabel("Número de Pixels")
    plt.hist(img_cinza.ravel(), bins=256, range=[0, 256])
    plt.xlim([0, 256])
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()

print("✅ Ambiente configurado e funções definidas com sucesso! (Versão Padronizada)")

imagens_carregadas = {}
imagem_selecionada_bgr = None
nome_arquivo_selecionado = ""
imagem_processada_final = None
download_lock = False

# Célula 2: Upload de Múltiplas Imagens


In [ ]:


imagens_carregadas = {}
imagem_selecionada_bgr = None
nome_arquivo_selecionado = ""

print("Por favor, selecione uma ou mais imagens (JPG, JPEG, PNG).")
uploaded = files.upload()

if uploaded:
    arquivos_validos = 0
    for nome_arquivo, dados_arquivo in uploaded.items():
        extensao = nome_arquivo.split('.')[-1].lower()

        if extensao in ['jpg', 'jpeg', 'png']:
            try:
                buffer_imagem = np.frombuffer(dados_arquivo, np.uint8)
                imagem_bgr = cv.imdecode(buffer_imagem, cv.IMREAD_COLOR)

                imagens_carregadas[nome_arquivo] = imagem_bgr
                arquivos_validos += 1

            except Exception as e:
                print(f"\n❌ Erro ao processar '{nome_arquivo}': Arquivo corrompido ou inválido.")
        else:
            print(f"\n❌ Erro: Arquivo '{nome_arquivo}' IGNORADO. Formato não suportado.")

    if arquivos_validos > 0:
        print(f"\n✅ {arquivos_validos} imagem(ns) carregada(s) com sucesso!")
        print("➡️ Execute a próxima célula para SELECIONAR com qual imagem trabalhar.")
    else:
        print("\nNenhuma imagem válida foi carregada.")
else:
    print("\nNenhuma imagem foi selecionada.")

# Célula 3: Exibição e Processamento Inicial para todas as imagens

In [ ]:
if imagens_carregadas:
    print(f"Exibindo a conversão padrão para as {len(imagens_carregadas)} imagens carregadas...")
    print("-" * 40)

    for nome_arquivo, imagem_bgr in imagens_carregadas.items():

        imagem_rgb = cv.cvtColor(imagem_bgr, cv.COLOR_BGR2RGB)

        imagem_cinza_padrao = converter_para_cinza(imagem_bgr)

        fig, axes = plt.subplots(1, 2, figsize=(12, 6))

        axes[0].imshow(imagem_rgb)
        axes[0].set_title(f'Original: {nome_arquivo}')
        axes[0].axis('off')

        axes[1].imshow(imagem_cinza_padrao, cmap='gray')
        axes[1].set_title('Convertida (Escala de Cinza)')
        axes[1].axis('off')

        plt.tight_layout()
        plt.show()

        altura, largura, _ = imagem_bgr.shape
        print(f"Dimensões: {largura}px (largura) x {altura}px (altura)")
        print("-" * 40)

else:
    print("❌ Nenhuma imagem carregada. Execute a Célula 2 para fazer o upload primeiro.")




# Célula 3.5: Seleção da Imagem Ativa

In [ ]:
if imagens_carregadas:
    seletor_imagem = widgets.Dropdown(
        options=list(imagens_carregadas.keys()),
        description='Imagem Ativa:',
        disabled=False,
    )

    output_selecao = widgets.Output()

    def ao_selecionar_imagem(change):
        global imagem_selecionada_bgr, nome_arquivo_selecionado

        nome_arquivo_selecionado = change['new']

        imagem_selecionada_bgr = imagens_carregadas[nome_arquivo_selecionado]

        with output_selecao:
            clear_output(wait=True)

            imagem_rgb = cv.cvtColor(imagem_selecionada_bgr, cv.COLOR_BGR2RGB)

            plt.figure(figsize=(7, 5))
            plt.imshow(imagem_rgb)
            plt.title(f"Visualizando: {nome_arquivo_selecionado}")
            plt.axis('off')
            plt.show()

            print("\n✅ Imagem selecionada! Prossiga para as próximas células para processá-la.")

    seletor_imagem.observe(ao_selecionar_imagem, names='value')

    print("Selecione a imagem que deseja processar:")
    display(seletor_imagem, output_selecao)
    ao_selecionar_imagem({'new': seletor_imagem.value})

else:
    print("❌ Nenhuma imagem carregada. Execute a Célula 2 para fazer o upload primeiro.")

# Célula 4: Painel de Controle Interativo

In [ ]:
if imagem_selecionada_bgr is not None:

    algoritmo_dropdown = widgets.Dropdown(
        options=['Nenhum', 'Logaritmo', 'Quadrado', 'Exponencial'],
        value='Nenhum',
        description='Transformação:',
        disabled=False,
    )

    inverter_checkbox = widgets.Checkbox(
        value=False,
        description='Inverter Cores (Negativo)',
        disabled=False
    )

    output_area = widgets.Output()


    def on_controls_change(change):
        global imagem_processada_final

        with output_area:
            clear_output(wait=True)

            img_cinza = converter_para_cinza(imagem_selecionada_bgr)

            metodo = algoritmo_dropdown.value
            img_transformada = aplicar_transformacao(img_cinza, metodo)

            if inverter_checkbox.value:
                img_transformada = inverter_cores(img_transformada)

            imagem_processada_final = img_transformada


            imagem_original_rgb = cv.cvtColor(imagem_selecionada_bgr, cv.COLOR_BGR2RGB)

            fig, axes = plt.subplots(1, 2, figsize=(14, 7))

            if len(img_transformada.shape) == 2:
                prova_canais_cinza = "1 Canal (Escala de Cinza)"
            else:
                prova_canais_cinza = f"{img_transformada.shape[2]} Canais"

            if len(imagem_original_rgb.shape) == 3:
                prova_canais_original = f"{imagem_original_rgb.shape[2]} Canais (RGB)"
            else:
                prova_canais_original = "1 Canal"

            axes[0].imshow(img_transformada, cmap='gray')
            axes[0].set_title(f"Imagem Processada (Canais: {prova_canais_cinza})")
            axes[0].axis('off')

            axes[1].imshow(imagem_original_rgb)
            axes[1].set_title(f"Imagem Original (Canais: {prova_canais_original})")
            axes[1].axis('off')

            plt.show()

            plotar_histograma(img_transformada)

    algoritmo_dropdown.observe(on_controls_change, names='value')
    inverter_checkbox.observe(on_controls_change, names='value')

    print("Use os controles abaixo para manipular a imagem.")
    display(widgets.VBox([
        widgets.HBox([algoritmo_dropdown, inverter_checkbox]),
        output_area
    ]))

    on_controls_change(None)

else:
    print("❌ Nenhuma imagem selecionada. Por favor, execute a Célula 2 (Upload) e depois a Célula 3.5 (Seleção) primeiro.")

#Celula 5: Painel de download

In [ ]:
clear_output(wait=True)

DOWNLOAD_EM_ANDAMENTO = False

if 'imagens_carregadas' in globals() and imagens_carregadas:

    def iniciar_download_em_lote_zip(b):
        global DOWNLOAD_EM_ANDAMENTO

        if DOWNLOAD_EM_ANDAMENTO:
            with output_download:
                 print("⚠️ Processo já em andamento! Por favor, aguarde a finalização.")
            return

        DOWNLOAD_EM_ANDAMENTO = True
        botao_download.disabled = True

        try:
            with output_download:
                clear_output(wait=True)
                print("🚀 Iniciando processo de compactação...")

                escolha = escolha_download.value
                metodo_atual = algoritmo_dropdown.value
                inverter_atual = inverter_checkbox.value
                nome_arquivo_zip = f"imagens_processadas_{int(time.time())}.zip"

                with zipfile.ZipFile(nome_arquivo_zip, 'w') as zf:
                    total = len(imagens_carregadas)
                    for i, (nome_arquivo, imagem_bgr) in enumerate(imagens_carregadas.items()):
                        print(f"[{i+1}/{total}] Processando: {nome_arquivo}...")
                        imagem_final = None
                        sufixo_nome = ""

                        if escolha == 'Aplicar Transformação Atual (em todas)':
                            sufixo_nome = f"_{metodo_atual.lower()}"
                            if inverter_atual:
                                sufixo_nome += "_invertido"
                            img_cinza = converter_para_cinza(imagem_bgr)
                            img_transformada = aplicar_transformacao(img_cinza, metodo_atual)
                            if inverter_atual:
                                img_transformada = inverter_cores(img_transformada)
                            imagem_final = img_transformada
                        else:
                            sufixo_nome = "_cinza"
                            imagem_final = converter_para_cinza(imagem_bgr)

                        if imagem_final is not None:
                            nome_base, extensao = os.path.splitext(nome_arquivo)

                            formato_saida = '.png' if extensao.lower() == '.png' else '.jpg'
                            nome_arquivo_saida = f"{nome_base}{sufixo_nome}{formato_saida}"

                            is_success, buffer_saida = cv.imencode(formato_saida, imagem_final)
                            if is_success:
                                zf.writestr(nome_arquivo_saida, buffer_saida.tobytes())
                                print(f"✔️ Adicionado '{nome_arquivo_saida}' ao ZIP.")
                            else:
                                print(f"❌ Falha ao codificar a imagem: {nome_arquivo}")

                print("\n" + "="*40)
                print(f"✅ Arquivo '{nome_arquivo_zip}' criado com sucesso! Iniciando download...")
                files.download(nome_arquivo_zip)

        finally:

            print("\nProcesso concluído! Liberando controles.")
            DOWNLOAD_EM_ANDAMENTO = False
            botao_download.disabled = False


    label_download = widgets.Label(value="Opção de Download em Lote:")
    escolha_download = widgets.RadioButtons(
        options=['Aplicar Transformação Atual (em todas)', 'Converter para Cinza Padrão (em todas)'],
        description='',
        disabled=False
    )
    botao_download = widgets.Button(
        description="Gerar e Baixar .ZIP",
        button_style='success',
        tooltip='Processar e baixar um arquivo ZIP com todas as imagens carregadas',
        icon='download'
    )
    output_download = widgets.Output()


    botao_download.on_click(iniciar_download_em_lote_zip)

    print("Selecione como deseja baixar TODAS as imagens carregadas e clique no botão.")
    box_opcoes = widgets.VBox([label_download, escolha_download])
    display(box_opcoes, botao_download, output_download)

else:
    print("❌ Nenhuma imagem carregada. Execute a Célula 2 (Upload) e a Célula 3.5 (Seleção) primeiro.")